In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/taeefnajib/used-car-price-prediction-dataset/used_cars.csv
/kaggle/input/current-9-14-2nd-place-solution/__results__.html
/kaggle/input/current-9-14-2nd-place-solution/submission.csv
/kaggle/input/current-9-14-2nd-place-solution/__notebook__.ipynb
/kaggle/input/current-9-14-2nd-place-solution/__output__.json
/kaggle/input/current-9-14-2nd-place-solution/submission_1.csv
/kaggle/input/current-9-14-2nd-place-solution/custom.css
/kaggle/input/current-9-14-2nd-place-solution/__results___files/__results___21_1.png
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/test_error.tsv
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/learn_error.tsv
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/catboost_training.json
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/time_left.tsv
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/learn/events.out.tfevents
/kaggle/input/current-9-14-2nd-place-solution/catboost_info/test/events.out

In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import re
from copy import deepcopy
from IPython.display import display_html, clear_output
from gc import collect

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, f1_score, r2_score, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline

from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor, log_evaluation, early_stopping
import lightgbm as lgb

import os

import warnings
warnings.filterwarnings("ignore")

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

from tqdm import tqdm, trange
from tabulate import tabulate
from sklearn.base import clone

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")
df_origin = pd.read_csv("/kaggle/input/used-car-price-prediction-dataset/used_cars.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s4e9/sample_submission.csv")

In [4]:
sub_1 = pd.read_csv("/kaggle/input/current-9-14-2nd-place-solution/submission.csv")

In [5]:
# drop columns which not used
df_train.drop(columns=["id"], inplace=True)
df_test.drop(columns=["id"], inplace=True)

df_origin[['milage', 'price']] = df_origin[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

df_train = pd.concat([df_train, df_origin], ignore_index=True)
df_train = df_train.drop_duplicates()

df_train.nunique()

brand             57
model           1898
model_year        34
milage          6652
fuel_type          7
engine          1146
transmission      62
ext_col          319
int_col          156
accident           2
clean_title        1
price           1569
dtype: int64

In [6]:
# df_train.copy(deep=True)

# Log transform the 'price' and 'milage' to reduce skewness
#df_train['milage'] = np.log1p(df_train['milage'])
#df_train['price'] = np.log1p(df_train['price'])

df_train['milage_log'] = np.log1p(df_train['milage'])
df_test['milage_log'] = np.log1p(df_test['milage'])

In [7]:
# Define a function to remove outliers using the IQR method
# def remove_outliers(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# df_train = remove_outliers(df_train, 'milage')
# df_train = remove_outliers(df_train, 'milage_log')
# df_train = remove_outliers(df_train, 'price')
# df_train = remove_outliers(df_train, 'price_log')

In [8]:
def extract_age_features(df):
    current_year = 2024

    df['Vehicle_Age'] = current_year - df['model_year']
    df['Mileage_per_Year'] = df['milage'] / df['Vehicle_Age']
    df['milage_with_age'] =  df.groupby('Vehicle_Age')['milage'].transform('mean')
    df['Mileage_per_Year_with_age'] =  df.groupby('Vehicle_Age')['Mileage_per_Year'].transform('mean')
    #df['milage_brand'] =  df.groupby('brand')['milage'].transform('mean')
    return df

In [9]:
def extract_other_features(df):
    luxury_brands =  ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land', 
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini', 
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']
    df['Is_Luxury_Brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)
    return df

In [10]:
def extract_engine_features(df):
    
    def extract_horsepower(engine):
        try:
            return float(engine.split('HP')[0])
        except:
            return None

    def extract_engine_size(engine):
        try:
            return float(engine.split(' ')[1].replace('L', ''))
        except:
            return None

    df['Horsepower'] = df['engine'].apply(extract_horsepower)
    df['Engine_Size'] = df['engine'].apply(extract_engine_size)
    df['Power_to_Weight_Ratio'] = df['Horsepower'] / df['Engine_Size']
    
    return df

In [11]:
def features_extraction(df):
#     df = extract_car_features(df)
    df = extract_engine_features(df)
#     df = extract_transmission_features(df)
    
    df = extract_age_features(df)
    df = extract_other_features(df)
    return df

In [12]:
%%time

train = features_extraction(df_train)
test = features_extraction(df_test)

CPU times: user 881 ms, sys: 5.83 ms, total: 886 ms
Wall time: 888 ms


In [13]:
train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,price,milage_log,Horsepower,Engine_Size,Power_to_Weight_Ratio,Vehicle_Age,Mileage_per_Year,milage_with_age,Mileage_per_Year_with_age,Is_Luxury_Brand
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,...,4200,12.269052,172.0,1.6,107.500000,17,12529.411765,118194.992360,6952.646609,0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,...,4999,11.872354,252.0,3.9,64.615385,22,6511.363636,118270.783105,5375.944687,0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,...,13900,11.825778,320.0,5.3,60.377358,22,6215.045455,118270.783105,5375.944687,0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,...,45000,9.878221,420.0,5.0,84.000000,7,2785.714286,67981.161419,9711.594488,0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,...,97500,8.907748,208.0,2.0,104.000000,3,2462.666667,29327.437406,9775.812469,1


In [14]:
def update(df):
    
    t = 100
    
    cat_c = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title',
             
            ]
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for col in re_:
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
        
    for col in cat_c:
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')
        
    return df

In [15]:
df_train['fuel_type'] = df_train['fuel_type'].replace({'–': 'unknown',
                                                         'Plug-In Hybrid':'Plug_In Hybrid'})
df_test['fuel_type'] =df_test['fuel_type'].replace({'–': 'unknown',
                                                       'Plug-In Hybrid':'Plug_In Hybrid'})

df_train['fuel_type'] = df_train['fuel_type'].fillna('unknown')
df_test['fuel_type'] =df_test['fuel_type'].fillna('unknown')

In [16]:
df_train['accident'] = df_train['accident'].fillna('None reported')
df_test['accident'] =df_test['accident'].fillna('None reported')

In [17]:
for col in ['Horsepower', 'Engine_Size', 'Power_to_Weight_Ratio']:
    imputer = SimpleImputer()
    imputer.fit(pd.concat([df_train[[col]], df_test[[col]]]))
    df_train[col] = imputer.transform(df_train[[col]])
    df_test[col] = imputer.transform(df_test[[col]])

In [18]:
train  = update(train)
test   = update(test)

X = train.drop('price', axis=1)
y = train['price']

In [19]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192542 entries, 0 to 192541
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   brand                      192542 non-null  category
 1   model                      192542 non-null  category
 2   model_year                 192542 non-null  int64   
 3   milage                     192542 non-null  int64   
 4   fuel_type                  192542 non-null  category
 5   engine                     192542 non-null  category
 6   transmission               192542 non-null  category
 7   ext_col                    192542 non-null  category
 8   int_col                    192542 non-null  category
 9   accident                   192542 non-null  category
 10  clean_title                192542 non-null  category
 11  price                      192542 non-null  int64   
 12  milage_log                 192542 non-null  float64 
 13  Horsepower    

(None, None)

In [20]:
import numpy as np
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=200)]

cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"cat_cols--------{cat_cols}")


def get_MAE_oof(df, target, lgb_params, cat_params=None, model_type='LGBM'):

    
    oof_predictions = np.zeros(len(df))
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    models = []
    rmse_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        print(f"Training fold {fold + 1}/{5} with {model_type}")

        X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]

        if model_type == 'LGBM':
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            model = lgb.train(
                lgb_params,
                train_data,
                valid_sets=[train_data, val_data],
                valid_names=['train', 'valid'],
                callbacks=callbacks    
            )
        
        elif model_type == 'CAT':
            train_data = Pool(data=X_train, label=y_train , cat_features=cat_cols)
            val_data = Pool(data=X_val, label=y_val , cat_features=cat_cols )
            
            model = CatBoostRegressor(**cat_params)
            model.fit(train_data, eval_set=val_data, verbose=150, early_stopping_rounds=200)
        
        models.append(model)
        
        if model_type == 'LGBM':
            pred = model.predict(X_val, num_iteration=model.best_iteration)
        elif model_type == 'CAT':
            pred = model.predict(X_val)
        
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        rmse_scores.append(rmse)

        print(f'{model_type} Fold RMSE: {rmse}')
        
        oof_predictions[val_idx] = pred
        
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    return oof_predictions, models


lgb_params = {
    'objective': 'MAE',
    'n_estimators': 1000,
    'random_state': 42,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')
X['LGBM_MAE'] = oof_predictions_lgbm


LGBM_preds = np.zeros(len(test))
for model in models_lgbm:
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MAE'] = LGBM_preds


lgb_params = {
    'objective': 'MSE',
    'n_estimators': 1000,
    'random_state': 42,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')

X['LGBM_MSE_diff'] = oof_predictions_lgbm - X['LGBM_MAE']

LGBM_preds = np.zeros(len(test))
for model in models_lgbm:
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MSE_diff'] = LGBM_preds - test['LGBM_MAE']

cat_cols--------['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
Training fold 1/5 with LGBM
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2506
[LightGBM] [Info] Number of data points in the train set: 154033, number of used features: 20
[LightGBM] [Info] Start training from score 30775.000000
Training until validation scores don't improve for 200 rounds
[300]	train's l1: 16244.6	valid's l1: 17233.8
[600]	train's l1: 16013.4	valid's l1: 17235.4
Early stopping, best iteration is:
[436]	train's l1: 16118.2	valid's l1: 17230.1
LGBM Fold RMSE: 76013.60430394212
Tr

In [21]:
lgb_params_1 ={

    'learning_rate': 0.017521301504983752,
    'max_depth': 42,
    'reg_alpha': 0.06876635751774487, 
    'reg_lambda': 9.738899198284985,
    'num_leaves': 131,
    'subsample': 0.2683765421728044,
    'colsample_bytree': 0.44346036599709887,
    'n_estimators': 1000,
    'random_state': 42,
    'extra_tree' : True,
    'verbose' : -1
}

lgb_params_2 ={
    
    "num_leaves": 426,
    "max_depth": 20,
    "learning_rate": 0.011353178352988012,
    "n_estimators": 884,
    "subsample": 0.5772552201954328,
    "colsample_bytree": 0.9164865430101521,
    "reg_alpha": 1.48699088003429e-06,
    "reg_lambda": 0.41539458543414265,
    "min_data_in_leaf": 73,
    "feature_fraction": 0.751673655170548,
    "bagging_fraction": 0.5120415391590843,
    "bagging_freq": 2,
    "min_child_weight": 0.017236362383443497,
    "cat_smooth": 54.81317407769262,
    "verbose" : -1,
    "boosting_type" : "gbdt"
}

cat_params={
    'learning_rate':0.042,
    'iterations':1000,
    'depth':10,
    'random_strength' : 10,
    'cat_features':cat_cols,
    'l2_leaf_reg':0.3,
    'random_seed':42,
    'early_stopping_rounds': 200,
    'task_type':'GPU',
}


kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scores = []
rmse_scores_cat = []
LGBM_model=[]
CAT_model =[]

callbacks = [log_evaluation(period=150), early_stopping(stopping_rounds=200)]

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    

    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    model_1 = lgb.train(lgb_params_1,
                      train_data,
                      valid_sets=[train_data, val_data],
                      valid_names=['train', 'valid'],
                      callbacks=callbacks        
                      )
    model_2 = lgb.train(lgb_params_2,
                  train_data,
                  valid_sets=[train_data, val_data],
                  valid_names=['train', 'valid'],
                  callbacks=callbacks        
                  )
    
    LGBM_model.append(model_1)
    LGBM_model.append(model_2)

    y_pred = model_1.predict(X_val) * 0.5 + model_2.predict(X_val) * 0.5

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    
    print(f'LGBM Fold RMSE: {rmse}')
    
    model_cat = CatBoostRegressor(**cat_params)

    train_pool = Pool(X_train, y_train ,cat_features=cat_cols)
    val_pool = Pool(X_val, y_val , cat_features=cat_cols)
    model_cat.fit(train_pool, eval_set=val_pool, verbose=300)
    
    CAT_model.append(model_cat)
    y_pred_cat = model_cat.predict(X_val)
    rmse_cat = np.sqrt(mean_squared_error(y_val, y_pred_cat))
    rmse_scores_cat.append(rmse_cat)
    
    print(f'CAT Fold RMSE: {rmse_cat}')

print(f'Mean LGBM RMSE: {np.mean(rmse_scores)}')
print(f'Mean CAT RMSE: {np.mean(rmse_scores_cat)}')

Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.03648e+09	valid's l2: 5.65447e+09
[300]	train's l2: 4.86104e+09	valid's l2: 5.64795e+09
Early stopping, best iteration is:
[224]	train's l2: 4.94581e+09	valid's l2: 5.64391e+09
Training until validation scores don't improve for 200 rounds
[150]	train's l2: 4.94799e+09	valid's l2: 5.65045e+09
[300]	train's l2: 4.67705e+09	valid's l2: 5.63223e+09
Early stopping, best iteration is:
[245]	train's l2: 4.75862e+09	valid's l2: 5.62889e+09
LGBM Fold RMSE: 75025.40528601382
0:	learn: 77814.9128042	test: 80563.9186664	best: 80563.9186664 (0)	total: 5.97s	remaining: 1h 39m 20s
300:	learn: 64896.6918453	test: 75188.6478543	best: 75069.1688545 (121)	total: 19s	remaining: 44s
bestTest = 75069.16885
bestIteration = 121
Shrink model to first 122 iterations.
CAT Fold RMSE: 75069.1685828319
Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.27991e+09	valid's l2: 4.61625e+09
[300]	train's l2: 

In [22]:
LGBM_preds = np.zeros(len(test))
for model in LGBM_model:
    LGBM_preds += model.predict(test) / len(LGBM_model)
    
CAT_preds = np.zeros(len(test))
for model in CAT_model:
    CAT_preds += model.predict(test) / len(CAT_model)
    
test_preds = LGBM_preds * 0.9 + CAT_preds * 0.1

In [23]:
submission['price'] = test_preds*0.5 + sub_1['price']*0.5
submission.to_csv("submission.csv", index=False)
submission.head()

,id,price
0,188533,18264.073134
1,188534,77510.175521
2,188535,57571.212814
3,188536,30342.111339
4,188537,30131.363867


In [24]:
feature_names = X.columns


lgb_importances = np.zeros(len(feature_names))

for model in LGBM_model:
    lgb_importances += model.feature_importance(importance_type='split') / len(LGBM_model)

lgb_feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': lgb_importances})
lgb_feature_importance.sort_values(by='Importance', ascending=False, inplace=True)


print(lgb_feature_importance)


                      Feature  Importance
21              LGBM_MSE_diff      9524.1
16           Mileage_per_Year      9191.4
3                      milage      8687.8
20                   LGBM_MAE      7554.2
12                 Horsepower      4463.4
6                transmission      3529.3
14      Power_to_Weight_Ratio      3489.7
0                       brand      3399.1
2                  model_year      2988.6
18  Mileage_per_Year_with_age      2638.9
7                     ext_col      2337.2
11                 milage_log      2323.8
5                      engine      2282.8
13                Engine_Size      2157.4
8                     int_col      1751.2
1                       model      1586.5
19            Is_Luxury_Brand      1269.8
9                    accident      1215.8
17            milage_with_age      1166.6
10                clean_title      1089.7
15                Vehicle_Age       879.5
4                   fuel_type       820.3


In [25]:
feature_names = X.columns
cat_importances = np.zeros(len(feature_names))

for model in CAT_model:
 
    cat_importances += model.get_feature_importance(type='FeatureImportance') / len(CAT_model)

cat_feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': cat_importances})
cat_feature_importance.sort_values(by='Importance', ascending=False, inplace=True)


print(cat_feature_importance)

                      Feature  Importance
20                   LGBM_MAE   16.554261
6                transmission    8.830768
0                       brand    8.590364
7                     ext_col    7.286914
8                     int_col    7.071206
5                      engine    5.337408
21              LGBM_MSE_diff    5.261986
16           Mileage_per_Year    4.957615
3                      milage    4.590167
1                       model    4.026456
4                   fuel_type    4.010080
15                Vehicle_Age    3.351186
19            Is_Luxury_Brand    3.333403
17            milage_with_age    2.693640
11                 milage_log    2.620792
2                  model_year    2.189146
10                clean_title    1.904119
18  Mileage_per_Year_with_age    1.681220
9                    accident    1.631336
12                 Horsepower    1.618871
13                Engine_Size    1.407507
14      Power_to_Weight_Ratio    1.051552


In [26]:
# %%time 

# class FtreMaker:
#     "This class makes new features using public discussions and kernel ideas"
    
#     def __init__(self):
#         self.fuel_type_dict = {
#             'Gasoline': 0,
#             'Hybrid': 1,
#             'E85 Flex Fuel': 2,
#             'uknown': 3,
#             'Diesel': 4,
#             'dash': 5,
#             'Plug-In Hybrid': 6,
#             'not supported': 7
#         }

#         self.accident_dict = {
#             'None reported': 0,
#             'At least 1 accident or damage reported': 1,
#             'uknown': 2
#         }

#         self.clean_title_dict = {
#             'Yes': 0,
#             'uknown': 1
#         }

#         self.expensive_ext_color = \
#         [  'Blue Caelum', 'Dark Sapphire', 'Bianco Monocerus', 'C / C', 'Ice',
#            'Tempest', 'Beluga Black', 'Bianco Icarus Metallic', 
#            'BLU ELEOS', 'Shadow Black', 'Nero Noctis', 'Sandstone Metallic',
#            'Lizard Green', 'Balloon White', 'Onyx', 'Donington Grey Metallic',
#            'China Blue', 'Diamond White', 'Rosso Corsa', 'Granite',
#            'Rosso Mars Metallic', 'Carpathian Grey', 'Kemora Gray Metallic',
#            'Grigio Nimbus', 'dash', 'Bianco Isis', 'Python Green', 'Fountain Blue',
#            'Custom Color', 'Vega Blue', 'Designo Magno Matte',
#            'Brands Hatch Gray Metallic', 'Rift Metallic', 'Gentian Blue Metallic',
#            'Arancio Borealis', 'BLUE', 'Aventurine Green Metallic', 'Apex Blue',
#            'Daytona Gray Pearl Effect', 'Daytona Gray Pearl Effect w/ Black Roof',
#            'Matte White', 'Carpathian Grey Premium Metallic', 'Blue Metallic',
#            'Santorini Black Metallic', 'Quartzite Grey Metallic',
#            'Carrara White Metallic', 'BLACK', 'Kinetic Blue', 'Nero Daytona'
#         ]

#         self.expensive_int_color = \
#         ['Dark Auburn', 'Hotspur', 'Cobalt Blue', 'Beluga Hide', 'Linen',
#          'Beluga', 'Black / Brown', 'Nero Ade', 'Sahara Tan', 'Portland']

#         self.expensive_hp = \
#         [443.0, 473.0, 493.0, 502.0, 521.0, 542.0, 
#          543.0, 571.0, 572.0, 573.0, 580.0,
#          591.0, 602.0, 611.0, 616.0, 620.0, 624.0, 
#          640.0, 641.0, 651.0, 710.0, 715.0, 760.0, 788.0, 797.0
#         ]
        
#         self.series_pattern = \
#         re.compile(r'^[A-Za-z0-9\-]+')
        
#         self.version_pattern = \
#         re.compile(r'([0-9]+\.[0-9]+[A-Za-z]*)|([A-Z]+[0-9]*)')
        
#         self.trim_pattern = \
#         re.compile(r'\b(Base|Sport|Premium|Ultimate|XLT|LZ|LT|Plus|Touring|SE|LE|Limited|Platinum|Performance|S|V6|GT|EX|SX|XLE|SR|SL|SV|XSE|TRD|RS|GranSport|Signature|Quad Cab|DRW|Cabriolet|Carbon Edition|Trail Boss|Prestige|Essence|Reserve|xDrive|4MATIC|PreRunner|EcoBoost|Scat Pack|Competition|Adventure Package|Laramie|Grand Touring|Long Range)\b')
        
#         self.hp_pattern = r'(\d+(\.\d+)?)HP'
#         self.displacement_pattern = r'(\d+\.\d+)L'
#         self.cylinder_pattern = r'(\d+) Cylinder'

#         self.engine_configs = {
#             'V6': 'V6', 
#             'V8': 'V8', 
#             'V10': 'V10', 
#             'V12': 'V12',
#             'Straight 6': 'Straight 6', 
#             'Flat 6': 'Flat 6', 
#             'I4': 'I4'
#         }

#         self.forced_induction = {'Turbo', 'Twin Turbo'}
#         self.valve_configs    = {'DOHC', 'SOHC', 'GDI', 'PDI'}
#         self.fuel_systems     = {'MPFI', 'GDI', 'PDI', 'TFSI'}
        
#     def fit(self, X: pd.DataFrame, y = None, **params):
#         return self
    
#     def _make_mdlparts(self, model: str):
#         "This method extracts model components"
        
#         series  = self.series_pattern.search(model)
#         version = self.version_pattern.search(model)
#         trim    = self.trim_pattern.search(model)

#         return {
#             'Series'  : series.group(0) if series else "uknown",
#             'Version' : version.group(0) if version else "uknown",
#             'Trim'    : trim.group(0) if trim else "uknown"
#         }
    
#     def _parse_engine(self, engine_desc):
#         "This metod processes the engine description feature and returns sub-parts"
        
#         int_default_na = -1
#         obj_default_na = "uknown"
        
#         features = {
#             'horsepower': int_default_na, 
#             'displacement': int_default_na, 
#             'cylinder_count': int_default_na,
#             'engine_configuration':obj_default_na, 
#             'forced_induction': obj_default_na,
#             'valve_configuration': obj_default_na, 
#             'fuel_system': obj_default_na, 
#             'mild_hybrid': False,
#         }
        
#         patterns = {
#             'horsepower'    : (self.hp_pattern, lambda x: float(x)),
#             'displacement'  : (self.displacement_pattern, lambda x: float(x)),
#             'cylinder_count': (self.cylinder_pattern, lambda x: int(x)),
#         }
        
#         for feature, (pattern, convert) in patterns.items():
#             match = re.search(pattern, engine_desc)
#             if match:
#                 try:
#                     features[feature] = convert(match.group(1))
#                 except ValueError:
#                     pass  
        
#         features['engine_configuration'] = \
#         next((config for config, name in self.engine_configs.items() if config in engine_desc), 
#              obj_default_na
#             )
        
#         features['forced_induction'] = \
#         next((induction for induction in self.forced_induction if induction in engine_desc),
#              obj_default_na
#             )
        
#         features['valve_configuration'] = \
#         next((valve for valve in self.valve_configs if valve in engine_desc), 
#              obj_default_na
#             )
        
#         features['fuel_system'] = \
#         next((system for system in self.fuel_systems if system in engine_desc), 
#              obj_default_na
#             )
        
#         features['mild_hybrid'] = 'Mild Electric Hybrid' in engine_desc
        
#         return features
    
#     def _get_gear_number(self, trans):
#         match = re.search(r'(\d+)[-\s]?Speed', trans)
#         return int(match.group(1)) if match else -1

#     def _get_transmission_type(self, trans):
#         if 'A/T' in trans or 'Automatic' in trans:
#             return 'Automatic'
#         elif 'M/T' in trans or 'Manual' in trans:
#             return 'Manual'
#         elif 'CVT' in trans:
#             return 'CVT'
#         elif 'DCT' in trans:
#             return 'DCT'
#         else:
#             return 'Other'

#     def _get_special_features(self, trans):
#         features = []
#         if 'Dual Shift Mode' in trans:
#             features.append('Dual Shift Mode')
#         if 'Auto-Shift' in trans:
#             features.append('Auto-Shift')
#         if 'Overdrive' in trans:
#             features.append('Overdrive')
#         if 'Electronically Controlled' in trans:
#             features.append('Electronically Controlled')
#         if 'Variable' in trans:
#             features.append('Variable')
#         return ', '.join(features) if features else "uknown"

#     def _get_transmission_dsg(self, trans):
#         if 'A/T' in trans:
#             return 'A/T'
#         elif 'M/T' in trans:
#             return 'M/T'
#         elif 'CVT' in trans:
#             return 'CVT'
#         elif 'DCT' in trans:
#             return 'DCT'
#         else:
#             return "uknown"

#     def transform(self, X: pd.DataFrame, y = None, **params):
#         "This method makes new features from the existing dataset"

#         df_train = X.copy()
#         print(f"\n\n---> Starting shape = {df_train.shape}")
        
#         print(f"---> Imputing category columns")
#         cat_ftre     = df_train.select_dtypes("object").columns
#         df_train[cat_ftre] = df_train[cat_ftre].fillna("uknown")
        
#         print(f"---> Performing -150 feature transforms")
#         df_train['fuel_type_encoded']   = df_train['fuel_type'].map(self.fuel_type_dict)
#         df_train['accident_encoded']    = df_train['accident'].map(self.accident_dict)
#         df_train['clean_title_encoded'] = df_train['clean_title'].map(self.clean_title_dict)
        
#         df_train['expensive_color_ext_encoded'] = \
#         df_train.ext_col.isin(self.expensive_ext_color).astype(int)
        
#         df_train['expensive_color_int_encoded'] = \
#         df_train.int_col.isin(self.expensive_int_color).astype(int)
        
#         df_train['expensive_hp'] = \
#         df_train['engine'].str.extract(r'(\d+\.?\d*)HP').\
#         astype(float).isin(self.expensive_hp).\
#         astype(int)
        
#         df_train['cylinder'] = \
#         df_train['engine'].str.extract(r'(\d+\.?\d*) Cylinder').astype(float) 
        
#         df_train['got_V'] = df_train['model'].str.extract(r'(\d+\.?\d*) V').notna().astype(int)
        
#         print(f"---> Making car model components")
#         mdlparts = df_train['model'].apply(self._make_mdlparts).apply(pd.Series)
#         df_train       = pd.concat([df_train, mdlparts], axis=1)
#         del mdlparts
        
#         print(f"---> Making engine components")
#         unique_engines  = df_train['engine'].unique()
#         engine_features = [self._parse_engine(engine) for engine in unique_engines]
#         engine_features = pd.DataFrame(engine_features, index = unique_engines).reset_index()
#         engine_features.rename(columns={'index': 'engine'}, inplace=True)
        
#         df_train = df_train.merge(engine_features, how = "left", on = "engine")
#         del engine_features,unique_engines
 
#         print(f"---> Making transmission components")
#         df_train['nb_gears']   = \
#         df_train['transmission'].apply(self._get_gear_number)
#         df_train['transmission_mode'] = \
#         df_train['transmission'].apply(self._get_transmission_type)
#         df_train['transmission_special']  = \
#         df_train['transmission'].apply(self._get_special_features)
#         df_train['transmission_dsg'] = \
#         df_train['transmission'].apply(self._get_transmission_dsg)
        
#         print(f"---> Imputing all numeric and categorical features at the end")
#         num_ftre = df_train.select_dtypes(include=[np.number]).columns
#         cat_ftre = df_train.select_dtypes(include=['object']).columns
        
#         df_train[num_ftre] = df_train[num_ftre].fillna(-1)
#         df_train[cat_ftre] = df_train[cat_ftre].fillna("uknown").astype("category")
        
# #       df_train["Source"] = df_train["Source"].astype(str)
  
#         print(f"\n---> Final shape = {df_train.shape}")
#         return df_train